In [5]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

In [62]:
fst = list(map(int, input("Строка матрицы с пробелами (например '1 2 3'):").split()))
scnd = list(map(int, input("Строка матрицы с пробелами (например '1 2 3'):").split()))
thrd = list(map(int, input("Строка матрицы с пробелами (например '1 2 3'):").split()))

C = np.array([fst, scnd, thrd])

In [25]:
C

array([[16,  7, 13, 10, 22],
       [ 6, 15, 12,  9,  4],
       [14, 11,  5, 17,  8]])

In [64]:
a1, a2, a3 = map(int, input("a1, a2, a3:").split())
b1, b2, b3, b4, b5 = map(int, input("b1, b2, b3, b4, b5:").split())

In [65]:
A = np.array([a1, a2, a3])
B = np.array([b1, b2, b3, b4, b5])
N = len(A)
M = len(B)

In [66]:
df = pd.DataFrame(C, columns=B, index=A)
df

,120,200,210,150,150
250,14,9,15,11,20
300,7,12,15,4,9
300,17,8,5,11,14


In [67]:
P = np.zeros_like(C)
C_ = C.copy()
A_ = A.copy()
B_ = B.copy()
i = 0
j = 0
while not np.all(C_ == 1_000_000):
    i, j = np.unravel_index(C_.argmin(), C_.shape)
    P[i][j] = min(A_[i], B_[j])
    if A_[i] <= B_[j]:
        C_[i].fill(1_000_000)
        B_[j] -= A_[i]
        A_[i] = 0
    else:
        C_[:, j].fill(1_000_000)
        A_[i] -= B[j]
        B_[j] = 0

plan = ""
if np.count_nonzero(P) == N + M - 1:
    plan = "невырожденным"
else:
    plan = "вырожденным"
print("Опорный план является", plan)

F = 0
for i in range(N):
    for j in range(M):
        F += C[i][j] * P[i][j]
print("план перевозки груза:", P, "F:", F, sep="\n")

Опорный план является невырожденным
план перевозки груза:
[[  0 110   0   0  50]
 [120   0   0 150  30]
 [  0  90 210   0   0]]
F:
5470


Оптимальный план перевозки груза

In [68]:
A_ub = np.array([[1 if i in range(j*M, M*(j+1)) else 0 for i in range(N*M)] for j in range(N)])
b_ub = A

b_eq = B
A_eq = np.array([[1 if i % M == j else 0 for i in range(N*M)] for j in range(M)])

res = linprog(C.flatten(), A_ub, b_ub, A_eq, b_eq, options={'disp': True})

X = res.x.round(3).reshape(N, M)
F_min = res.fun.round(3)

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 171.0               
0.1107216986547     0.1107216986547     0.1107216986548     0.8963410867654  0.1107216986547     1300.84320416       
0.03652254215735    0.03652254215735    0.03652254215738    0.6942961629146  0.03652254215739    2381.261299701      
0.009387029673161   0.00938702967316    0.009387029673168   0.749969156504   0.009387029673169   4488.867000812      
0.002108734863103   0.002108734863105   0.002108734863107   0.8042965147856  0.002108734863135   5770.759366915      
0.0001299340030438  0.000129934003044   0.0001299340030442  0.9456893416472  0.0001299340030462  6264.090145598      
1.469443056102e-08  1.469443055045e-08  1.469443076779e-08  0.9998967549211  1.469443074116e-08  6299.995950404      
7.349267205539e-13  7.34927092007e-13   7.348333827733e-1

In [70]:
print("оптимальны план перевозки груза:", X, "F_min:", F_min, sep="\n")

оптимальны план перевозки груза:
[[ 16.907 200.      0.     13.093   0.   ]
 [103.093   0.      0.    136.907  60.   ]
 [  0.      0.    210.      0.     90.   ]]
F_min:
6300.0
